# 本文件说明
- 获取银保监会带机构名称的句子

# 基本设置

In [1]:
import numpy as np
import pandas as pd

import os

import requests,json
from sklearn.externals import joblib

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
from toolkits.setup.date_time import get_day_list
from toolkits.setup import specific_func

from toolkits.nlp import pre_cor_circ
from toolkits.nlp import pre_cor_cbrc
from toolkits.nlp import circ_NER

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.063 seconds.
Prefix dict has been built succesfully.


In [3]:
%load_ext autoreload
%autoreload 2

## 一些函数

# 保监会

## 读取数据

In [74]:
gather_types = '采集'

day_list = get_day_list('2018-12-07', '2018-12-10')
print(day_list)

['2018-12-08', '2018-12-09', '2018-12-10']


In [75]:
combined_data = pd.DataFrame()
for day_select in day_list:
    file_name = 'circ_result_class/result/%s_circ_class_predict_mysql_%s.xlsx'%(gather_types, day_select)
    if os.path.isfile(file_name):
        print(file_name)
        tmp_data = pd.read_excel(file_name)
        combined_data = pd.concat([combined_data, tmp_data], axis = 0)

if gather_types != '补录':
    combined_data = combined_data[combined_data['predict_label'] != '补录']
print(combined_data.shape)  
print('去重前：', combined_data.shape)
combined_data = combined_data.drop_duplicates(subset = 'title')
print('去重后：', combined_data.shape)  
combined_data = combined_data.dropna(subset = ['content'], axis = 0)
print('去空值后：', combined_data.shape)  

print(combined_data['predict_label'].value_counts())
combined_data.head()

circ_result_class/result/采集_circ_class_predict_mysql_2018-12-08.xlsx
circ_result_class/result/采集_circ_class_predict_mysql_2018-12-09.xlsx
circ_result_class/result/采集_circ_class_predict_mysql_2018-12-10.xlsx
(42059, 7)
去重前： (42059, 7)
去重后： (40184, 7)
去空值后： (40176, 7)
噪音        27295
资本市场       3120
监管         2310
消费服务       1942
其他相关报道     1662
产品销售       1473
行业         1287
公司内部管理     1087
Name: predict_label, dtype: int64


,id,group_id,predict_label,label,title,content,publishtime
0,12679251,微信,行业,NaN,周延礼：协同构建保险科技的新生态系统,12月5日，以“防范金融风险 维护金融安全”为主题的2018年北京金融安全论坛举行。全国政协...,2018-12-08 15:25:23
1,12657194,新闻,资本市场,NaN,中国人寿(601628)融资融券信息(12-07),中国人寿(601628)融资融券信息(12-07)2018年12月08日 07:39来源：东...,2018-12-08 04:20:14
2,12677044,微博,消费服务,NaN,#普法判例#【交通事故受害人用医保卡支付医疗费 保险公司可否拒赔？！】 ​,#普法判例#【交通事故受害人用医保卡支付医疗费 保险公司可否拒赔？！】 ​,2018-12-08 11:12:54
3,12657807,新闻,消费服务,NaN,从申请到结案“8秒” 太平人寿“秒赔”就是快,客户李女士在太平人寿黑龙江分公司为自己投保了爱无忧系列意外伤害保险。2018年9月27日，李...,2018-12-08 08:57:00
4,12654634,微信,消费服务,NaN,茂名某学生在饭堂发生意外，诉状学校、保险公司，结果.....,学生在饭堂就餐时摔伤 法院判决学校和保险公司担责 近日，信宜法院执结了一起人身损...,2018-12-08 00:30:12


In [6]:
combined_data_sel = combined_data[~combined_data['predict_label'].isin(['噪音'])][['id', 'title', 'content']]
combined_data_sel['title_content'] = combined_data_sel['title'].astype(str) + "。" + combined_data_sel['content'].astype(str)
print(combined_data_sel.shape)  
combined_data_sel.iloc[:1, :]

(27427, 4)


,id,title,content,title_content
0,12679251,周延礼：协同构建保险科技的新生态系统,12月5日，以“防范金融风险 维护金融安全”为主题的2018年北京金融安全论坛举行。全国政协...,周延礼：协同构建保险科技的新生态系统。12月5日，以“防范金融风险 维护金融安全”为主题的2...


## 获取句子数据

In [109]:
circ_sentences = []

for index, id in enumerate(combined_data_sel.loc[:, 'id']):
#     if index <1000:
    print('-------')
    print('index: ', index, 'id: ', id)
    content = combined_data_sel[combined_data_sel['id'] == id]['title_content'].tolist()[0]
#     print(content)

    content = circ_NER.clear_article(str(content))  # clear_article
#     print(content)
    sentences = [i.strip() for i in circ_NER.cut_sentences(content)]
#     print(sentences)
    pre_sentences = circ_NER.preprocess_sentences(sentences) 
    print('  len(pre_sentences): ', len(pre_sentences))
#         pre_sentences

    if len(pre_sentences) > 0:
        s_l_repeat = []
        org_list, org_loc = circ_NER.get_entity(pre_sentences, sentences)     
        if org_loc:
#             print('  len(org_loc): ', len(org_loc))
            for key, value in org_loc.items():
#                     print(key, value)
                for s_l in value[1]:
                    if s_l not in s_l_repeat:
#                         print(s_l, sentences[s_l])
                        circ_sentences.append([id, value[0], key,
                                               s_l, sentences[s_l]])   # id, name, aka_name, sen_loc, sentence
                        s_l_repeat.append(s_l)
                    

-------
index:  0 id:  12679251
  len(pre_sentences):  99
-------
index:  1 id:  12657194
  len(pre_sentences):  3
decide_sen:  -3
-------
index:  2 id:  12677044
  len(pre_sentences):  4
-------
index:  3 id:  12657807
  len(pre_sentences):  13
-------
index:  4 id:  12654634
  len(pre_sentences):  20
-------
index:  5 id:  12759888
  len(pre_sentences):  2
-------
index:  6 id:  12659115
  len(pre_sentences):  6
-------
index:  7 id:  12657178
  len(pre_sentences):  12
-------
index:  8 id:  12666334
  len(pre_sentences):  41
-------
index:  9 id:  12675770
  len(pre_sentences):  21
-------
index:  10 id:  12667320
  len(pre_sentences):  68
-------
index:  11 id:  12670938
  len(pre_sentences):  12
-------
index:  12 id:  12663453
  len(pre_sentences):  3
decide_sen:  -3
-------
index:  13 id:  12744128
  len(pre_sentences):  5
decide_sen:  -1
-------
index:  14 id:  12657803
  len(pre_sentences):  9
-------
index:  15 id:  12657535
  len(pre_sentences):  20
-------
index:  16 id:  1

  len(pre_sentences):  2
-------
index:  125 id:  12665357
  len(pre_sentences):  27
-------
index:  126 id:  12681653
  len(pre_sentences):  6
-------
index:  127 id:  12666789
  len(pre_sentences):  25
-------
index:  128 id:  12658843
  len(pre_sentences):  7
-------
index:  129 id:  12682063
  len(pre_sentences):  11
-------
index:  130 id:  12667874
  len(pre_sentences):  25
-------
index:  131 id:  12659589
  len(pre_sentences):  42
-------
index:  132 id:  12679241
  len(pre_sentences):  18
-------
index:  133 id:  12667155
  len(pre_sentences):  13
-------
index:  134 id:  12668093
  len(pre_sentences):  36
-------
index:  135 id:  12676526
  len(pre_sentences):  58
-------
index:  136 id:  12657320
  len(pre_sentences):  2
-------
index:  137 id:  12698295
  len(pre_sentences):  81
-------
index:  138 id:  12657561
  len(pre_sentences):  15
decide_sen:  -2
-------
index:  139 id:  12655788
  len(pre_sentences):  48
-------
index:  140 id:  12668748
  len(pre_sentences):  29
--

  len(pre_sentences):  4
-------
index:  248 id:  12717789
  len(pre_sentences):  10
-------
index:  249 id:  12657422
  len(pre_sentences):  62
-------
index:  250 id:  12657245
  len(pre_sentences):  32
-------
index:  251 id:  12657260
  len(pre_sentences):  87
decide_sen:  -4
-------
index:  252 id:  12665358
  len(pre_sentences):  12
-------
index:  253 id:  12667397
  len(pre_sentences):  49
decide_sen:  -4
decide_sen:  -2
decide_sen:  -4
-------
index:  254 id:  12660808
  len(pre_sentences):  9
decide_sen:  -4
-------
index:  255 id:  12712732
  len(pre_sentences):  4
-------
index:  256 id:  12659559
  len(pre_sentences):  98
-------
index:  257 id:  12659816
  len(pre_sentences):  29
decide_sen:  -3
-------
index:  258 id:  12676175
  len(pre_sentences):  52
-------
index:  259 id:  12655579
  len(pre_sentences):  36
decide_sen:  -3
-------
index:  260 id:  12663160
  len(pre_sentences):  14
decide_sen:  -3
-------
index:  261 id:  12656945
  len(pre_sentences):  43
-------
i

index:  378 id:  12660124
  len(pre_sentences):  16
decide_sen:  -1
-------
index:  379 id:  12659219
  len(pre_sentences):  7
decide_sen:  -4
-------
index:  380 id:  12659099
  len(pre_sentences):  24
-------
index:  381 id:  12654825
  len(pre_sentences):  12
decide_sen:  -3
-------
index:  382 id:  12667335
  len(pre_sentences):  18
-------
index:  383 id:  12664434
  len(pre_sentences):  12
-------
index:  384 id:  12657801
  len(pre_sentences):  83
-------
index:  385 id:  12655005
  len(pre_sentences):  33
-------
index:  386 id:  12661933
  len(pre_sentences):  27
-------
index:  387 id:  12681767
  len(pre_sentences):  2
-------
index:  388 id:  12654451
  len(pre_sentences):  3
decide_sen:  -3
-------
index:  389 id:  12679455
  len(pre_sentences):  26
-------
index:  390 id:  12729586
  len(pre_sentences):  77
-------
index:  391 id:  12658624
  len(pre_sentences):  11
-------
index:  392 id:  12659395
  len(pre_sentences):  9
-------
index:  393 id:  12667792
  len(pre_sent

index:  501 id:  12667053
  len(pre_sentences):  66
decide_sen:  -3
-------
index:  502 id:  12742583
  len(pre_sentences):  13
decide_sen:  -4
-------
index:  503 id:  12655972
  len(pre_sentences):  2
-------
index:  504 id:  12664813
  len(pre_sentences):  16
-------
index:  505 id:  12663074
  len(pre_sentences):  17
decide_sen:  -3
decide_sen:  -4
decide_sen:  -4
-------
index:  506 id:  12660348
  len(pre_sentences):  38
decide_sen:  -4
decide_sen:  -4
decide_sen:  -4
decide_sen:  -4
decide_sen:  -4
decide_sen:  -4
decide_sen:  -4
-------
index:  507 id:  12663022
  len(pre_sentences):  6
decide_sen:  -3
decide_sen:  -4
-------
index:  508 id:  12665101
  len(pre_sentences):  22
-------
index:  509 id:  12660581
  len(pre_sentences):  18
-------
index:  510 id:  12664820
  len(pre_sentences):  39
decide_sen:  -4
-------
index:  511 id:  12664251
  len(pre_sentences):  16
-------
index:  512 id:  12661591
  len(pre_sentences):  11
-------
index:  513 id:  12656984
  len(pre_senten

...

index:  26834 id:  12764118
  len(pre_sentences):  226
-------
index:  26835 id:  12761102
  len(pre_sentences):  2
-------
index:  26836 id:  12746458
  len(pre_sentences):  612
-------
index:  26837 id:  12759581
  len(pre_sentences):  18
-------
index:  26838 id:  12766878
  len(pre_sentences):  68
decide_sen:  -4
-------
index:  26839 id:  12761928
  len(pre_sentences):  6
decide_sen:  -3
-------
index:  26840 id:  12761854
  len(pre_sentences):  15
-------
index:  26841 id:  12751026
  len(pre_sentences):  18
-------
index:  26842 id:  12761045
  len(pre_sentences):  13
-------
index:  26843 id:  12758603
  len(pre_sentences):  2
decide_sen:  -2
-------
index:  26844 id:  12759064
  len(pre_sentences):  10
-------
index:  26845 id:  12761100
  len(pre_sentences):  16
decide_sen:  -2
-------
index:  26846 id:  12748147
  len(pre_sentences):  6
-------
index:  26847 id:  12759386
  len(pre_sentences):  243
decide_sen:  -4
-------
index:  26848 id:  12768925
  len(pre_sentences):  12

  len(pre_sentences):  15
-------
index:  26959 id:  12761862
  len(pre_sentences):  62
-------
index:  26960 id:  12780568
  len(pre_sentences):  16
decide_sen:  -4
-------
index:  26961 id:  12748270
  len(pre_sentences):  18
-------
index:  26962 id:  12750358
  len(pre_sentences):  8
-------
index:  26963 id:  12758971
  len(pre_sentences):  5
-------
index:  26964 id:  12761238
  len(pre_sentences):  14
-------
index:  26965 id:  12755802
  len(pre_sentences):  81
-------
index:  26966 id:  12765000
  len(pre_sentences):  5
decide_sen:  -3
-------
index:  26967 id:  12759755
  len(pre_sentences):  10
-------
index:  26968 id:  12766446
  len(pre_sentences):  34
decide_sen:  -3
-------
index:  26969 id:  12751506
  len(pre_sentences):  19
-------
index:  26970 id:  12766470
  len(pre_sentences):  22
-------
index:  26971 id:  12759549
  len(pre_sentences):  57
-------
index:  26972 id:  12760401
  len(pre_sentences):  4
-------
index:  26973 id:  12747442
  len(pre_sentences):  8
d

-------
index:  27085 id:  12751404
  len(pre_sentences):  22
-------
index:  27086 id:  12758162
  len(pre_sentences):  14
-------
index:  27087 id:  12763691
  len(pre_sentences):  7
-------
index:  27088 id:  12785281
  len(pre_sentences):  6
-------
index:  27089 id:  12750766
  len(pre_sentences):  25
-------
index:  27090 id:  12764810
  len(pre_sentences):  8
decide_sen:  -3
-------
index:  27091 id:  12759776
  len(pre_sentences):  12
-------
index:  27092 id:  12768700
  len(pre_sentences):  10
-------
index:  27093 id:  12766489
  len(pre_sentences):  4
decide_sen:  -2
-------
index:  27094 id:  12745391
  len(pre_sentences):  32
-------
index:  27095 id:  12760220
  len(pre_sentences):  6
-------
index:  27096 id:  12754911
  len(pre_sentences):  32
-------
index:  27097 id:  12758982
  len(pre_sentences):  41
-------
index:  27098 id:  12747232
  len(pre_sentences):  14
-------
index:  27099 id:  12768953
  len(pre_sentences):  11
-------
index:  27100 id:  12761292
  len(p

-------
index:  27204 id:  12746383
  len(pre_sentences):  19
-------
index:  27205 id:  12750326
  len(pre_sentences):  23
-------
index:  27206 id:  12782316
  len(pre_sentences):  2
-------
index:  27207 id:  12773781
  len(pre_sentences):  20
-------
index:  27208 id:  12749562
  len(pre_sentences):  38
-------
index:  27209 id:  12769405
  len(pre_sentences):  14
decide_sen:  -4
decide_sen:  -4
-------
index:  27210 id:  12757988
  len(pre_sentences):  2
decide_sen:  -3
-------
index:  27211 id:  12749841
  len(pre_sentences):  8
decide_sen:  -3
-------
index:  27212 id:  12762383
  len(pre_sentences):  5
-------
index:  27213 id:  12750641
  len(pre_sentences):  4
-------
index:  27214 id:  12753250
  len(pre_sentences):  59
-------
index:  27215 id:  12763517
  len(pre_sentences):  196
-------
index:  27216 id:  12767860
  len(pre_sentences):  43
-------
index:  27217 id:  12786675
  len(pre_sentences):  19
-------
index:  27218 id:  12775048
  len(pre_sentences):  61
decide_sen

  len(pre_sentences):  47
-------
index:  27323 id:  12758714
  len(pre_sentences):  7
-------
index:  27324 id:  12752059
  len(pre_sentences):  25
-------
index:  27325 id:  12776242
  len(pre_sentences):  77
decide_sen:  -4
decide_sen:  -4
decide_sen:  -4
-------
index:  27326 id:  12765208
  len(pre_sentences):  28
-------
index:  27327 id:  12764253
  len(pre_sentences):  10
-------
index:  27328 id:  12752282
  len(pre_sentences):  13
-------
index:  27329 id:  12761984
  len(pre_sentences):  30
-------
index:  27330 id:  12774626
  len(pre_sentences):  26
decide_sen:  -4
-------
index:  27331 id:  12757709
  len(pre_sentences):  16
-------
index:  27332 id:  12782949
  len(pre_sentences):  16
decide_sen:  -4
-------
index:  27333 id:  12750392
  len(pre_sentences):  74
-------
index:  27334 id:  12747741
  len(pre_sentences):  6
-------
index:  27335 id:  12760333
  len(pre_sentences):  75
-------
index:  27336 id:  12766571
  len(pre_sentences):  12
-------
index:  27337 id:  1

In [110]:
# combined_data_sel.loc[12673139, 'title_content']#.tolist()[0]

In [121]:
circ_sentences_pd = pd.DataFrame(circ_sentences, columns = ['id', 'name', 'aka_name', 'sen_loc', 'sentence'])
circ_sentences_pd.insert(4, 'label', '')
print('去重前： ', circ_sentences_pd.shape)
circ_sentences_pd = circ_sentences_pd.drop_duplicates(subset = 'sentence')
print('去重后： ', circ_sentences_pd.shape)
# circ_sentences_pd = circ_sentences_pd.drop_duplicates(subset = 'id')
# print('去重后： ', circ_sentences_pd.shape)
# circ_sentences_pd = circ_sentences_pd[~circ_sentences_pd['name'].isin(['历任会领导', '各部领导'])]
# print('去重后： ', circ_sentences_pd.shape)
circ_sentences_pd.head()

去重前：  (58185, 6)
去重后：  (38244, 6)


,id,name,aka_name,sen_loc,label,sentence
0,12679251,历任会领导,周延礼,0,,周延礼：协同构建保险科技的新生态系统。
1,12679251,历任会领导,周延礼,3,,全国政协委员、原中国保监会副主席周延礼出席论坛并发表主题演讲。
2,12679251,历任会领导,周延礼,5,,以下为周延礼演讲内容摘要： 一、当前保险科技发展面临机遇 金融科技产生与发展对金融市场、金融...
3,12657194,中国人寿保险（集团）公司,中国人寿,0,,中国人寿(601628)融资融券信息(12-07)。
4,12657194,中国人寿保险（集团）公司,中国人寿,2,,中国人寿融资融券详细信息如下表：交易日期代码简称融资融券余额(元)2018-12-07601...


In [122]:
circ_sentences_pd.loc[:38000, :].to_excel('circ_result_tendency/circ_sentences_tendency_20181224(1208-1210).xlsx', 
                          index = False)